# Install dependencies

In [1]:
! pip install huggingface_hub

! pip install "/kaggle/input/efficient-llm-inference-wheels/bitsandbytes-0.40.0.post4-py3-none-any.whl"
! pip install "/kaggle/input/efficient-llm-inference-wheels/accelerate-0.20.3-py3-none-any.whl"
! pip install "/kaggle/input/efficient-llm-inference-wheels/peft-0.3.0-py3-none-any.whl"

! pip install evaluate>=0.30
! pip install jiwer
! pip install datasets==2.13.1
! pip install -U neptune transformers
! pip install neptune
! pip install accelerate bitsandbytes
!pip install -q bitsandbytes datasets accelerate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

Processing /kaggle/input/efficient-llm-inference-wheels/bitsandbytes-0.40.0.post4-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/efficient-llm-inference-wheels/bitsandbytes-0.40.0.post4-py3-none-any.whl'

Processing /kaggle/input/efficient-llm-inference-wheels/accelerate-0.20.3-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/efficient-llm-inference-wheels/accelerate-0.20.3-py3-none-any.whl'

Processing /kaggle/input/efficient-llm-inference-wheels/peft-0.3.0-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/efficient-llm-inference-wheels/peft-0.3.0-py3-none-any.whl'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of

# Importing Libraries

In [2]:
from datasets import load_dataset
import evaluate
from IPython.display import Audio as Play_audio
from types import SimpleNamespace

from sklearn.model_selection import train_test_split
import torch
import librosa
from tqdm import tqdm
from datasets import IterableDataset
from dataclasses import dataclass
import librosa
import torch
import numpy as np
from scipy.signal import resample
from typing import Any, Dict, List, Union
from transformers import (
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
)
from transformers.integrations import NeptuneCallback
import neptune

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import os

os.environ["project"] = "petr.square/Voice-Recognition-UA-KROK"
os.environ["api_token"] = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIwMjlhMTk5NS04ODcwLTQ3MzgtYmVlYy0xZWNkM2NkZGQwMmYifQ=="
os.environ["WANDB_PROJECT"] = "Voice-Recognition-UA-KROK"
os.environ["WANDB_API_KEY"] = "bff47a187222bed4cdccb1eb5efdc01f871ce829"

In [4]:
import pandas as pd
from datasets import Dataset

base_path = "/kaggle/input/krok-ua/Structured_Audio_Segments"
train_df = pd.read_excel(f"{base_path}/anotations.xlsx")

In [5]:
import os

existing_files = []
for index, row in tqdm(train_df.iterrows()):
    user_id = row["user_id"]
    sample_id = row["index"]
    file_path = os.path.join(base_path, str(user_id), f"{sample_id}.wav")
    if os.path.exists(file_path):
        existing_files.append(index)

1855it [00:02, 729.34it/s]


In [6]:
! pip install icecream

from icecream import ic as print

In [7]:
print(len(existing_files), len(train_df))
# train_df.loc[~train_df.index.isin(existing_files)]
train_df = train_df.loc[train_df.index.isin(existing_files)]
print(train_df.shape);

ic| len(existing_files): 1832, len(train_df): 1855
ic| train_df.shape: (1832, 3)


In [8]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("arampacha/whisper-large-uk-2")
tokenizer = WhisperTokenizer.from_pretrained("arampacha/whisper-large-uk-2", language="ukrainian", task="transcribe")
processor = WhisperProcessor.from_pretrained("arampacha/whisper-large-uk-2", language="ukrainian", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
train_sample, eval_sample = train_test_split(train_df, test_size=0.15, random_state=42)
sampling_rate = 16000
path_template = "/kaggle/input/krok-ua/Structured_Audio_Segments/{user_id}/{sample_id}.wav"

In [10]:
from datasets import IterableDataset

def dataset_generator(df):
    for _, row in df.iterrows():
        audio_array = librosa.load(path_template.format(
            user_id=row["user_id"],
            sample_id=row["index"]
        ))[0]
        yield {
            "input_features": feature_extractor(audio_array, sampling_rate=sampling_rate).input_features[0], 
            "labels": tokenizer(row["description"]).input_ids
        }

train_ds = IterableDataset.from_generator(dataset_generator, gen_kwargs={"df": train_sample})
eval_ds = IterableDataset.from_generator(dataset_generator, gen_kwargs={"df": eval_sample})

In [11]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [12]:
from peft import prepare_model_for_int8_training
from transformers import WhisperForConditionalGeneration

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

model_id = "arampacha/whisper-large-uk-2"
model = WhisperForConditionalGeneration.from_pretrained(model_id, load_in_8bit=True)

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

model = prepare_model_for_int8_training(model)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [13]:
import bitsandbytes as bnb

def find_all_linear_names(model, bits=8):
    cls = bnb.nn.Linear4bit if bits == 4 else (bnb.nn.Linear8bitLt if bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])


    if 'lm_head' in lora_module_names: # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [14]:
target_modules = find_all_linear_names(model)
target_modules

['k_proj', 'q_proj', 'fc2', 'fc1', 'v_proj', 'out_proj']

In [15]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=target_modules, lora_dropout=0.1, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 57,671,680 || all params: 1,600,976,640 || trainable%: 3.602281167575312


In [16]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-largev2-bn",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-4,
    warmup_steps=30,
    max_steps=400,
#     gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=4,
    generation_max_length=225,
    save_steps=30,
    eval_steps=30,
    logging_steps=5,
    report_to="wandb",
    load_best_model_at_end=True,
#     metric_for_best_model="wer",
#     greater_is_better=False,
    save_total_limit=5,
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
)

In [17]:
# metric = evaluate.load("wer")

# def compute_metrics(pred):
#     pred_ids = pred.predictions
#     label_ids = pred.label_ids

#     # replace -100 with the pad_token_id
#     label_ids[label_ids == -100] = tokenizer.pad_token_id

#     # we do not want to group tokens when computing the metrics
#     pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

#     wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    
#     return {"wer": wer}

In [18]:
import os
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR


class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
#     compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False

In [19]:
with torch.autocast("cuda"):
    trainer.train()

wandb: Currently logged in as: petr-square (bird-speech-clf). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231114_080521-jadx9o79
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run wise-smoke-21
wandb: ⭐️ View project at https://wandb.ai/bird-speech-clf/Voice-Recognition-UA-KROK
wandb: 🚀 View run at https://wandb.ai/bird-speech-clf/Voice-Recognition-UA-KROK/runs/jadx9o79
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 d

Step,Training Loss,Validation Loss
30,0.624900,0.462172
60,0.638500,0.388560
90,0.513500,0.358260
120,0.393400,0.345723
150,0.424300,0.331490
180,0.432200,0.328937
210,0.347600,0.330070
240,0.330900,0.328169
270,0.319900,0.322864
300,0.296700,0.318154


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to fl

In [20]:
trainer.save_model("best_model_bath")